# Data access and discovery

## Authors & Contributors

### Authors

- Pier Lorenzo Marasco, Ispra (Italy), [@pl-marasco](https://github.com/pl-marasco)
- Alejandro Coca-Castro, The Alan Turing Institute (United Kingdom), [@acocac](https://github.com/acocac)

### Contributors
- Tina Odaka, Ifremer (France), [@tinaok](https://github.com/tinaok)

<div class="alert alert-info">
    <i class="fa-question-circle fa" style="font-size: 22px;color:#666;"></i> <b>Overview</b>
    <br>
    <br>
    <b>Questions</b>
    <ul>
        <li>How to access online (remote) datasets?</li>
        <li>How to prepare and discover online geoscience datasets?</li>
        <li>What is Analysis Ready, Cloud Optimized data (ARCO)?</li>
        <li>What is pangeo-forge?</li>
        <li>What is stac?</li>
    </ul>
    <b>Objectives</b>
    <ul>
        <li>Learn to access datasets from online object storage</li>
        <li>Learn about preparing and discovery online datasets</li>
        <li>Learn about Analysis Cloud Optimized (ARCO) data</li>
        <li>Learnn about the Pangeo Forge initiative</li>
        <li>Learn about stac</li>
    </ul>
</div>

## Context

We will be using Long Term Statistics (1999-2019) product provided by the [Copernicus Global Land Service over Lombardia](https://land.copernicus.eu/global/index.html) and access them through [S3-compatible storage](https://en.wikipedia.org/wiki/Amazon_S3). Also we will explore [Sentinel-2 Cloud-Optimised Dataset](https://registry.opendata.aws/sentinel-2-l2a-cogs/) online through [STAC](https://stacspec.org/en).

## Setup

This episode uses the following Python packages:

- xarray
- s3fs
- datetime
- pystac_client

Please install these packages if not already available in your Python environment. Below, we only install packages that are not available in the EGI-ACE EOSC deployment of Pangeo for the FOSS4G course.

### Packages

In this episode, Python packages are imported when we start to use them. However, for best software practices, we recommend you to install and import all the necessary libraries at the top of your Jupyter notebook.

## Introduction to the Long Term statistics
CGLS LTS are computed over a time span of 20 years aggregated over separate 10-day periods (month/01,month/11, month/21). For each date the long term minimum, maximum, mean, median and standard deviation are computed.

### S3-compatible Object Storage to access online data

Up to now we downloaded data locally and then opened the downloaded file with Xarray `open_dataset`. Each attendee downloaded a copy locally. When willing to manipulate large amount of data, this approach is not optimal (since it requires a lot of unecessary local downloads). Sharing data online as Object Storage allows for data sharing and access to much larger amounts of data.

One of the most popular ways to access online remote data is through Amazon Simple Storage Service (S3) and you do not necessarily need to use Amazon services to benefit from S3 Object storage. Many other providers offer S3-compatible object storage that can be accessed in a very similar way.

Below we will be accessing online the NDVI Long Term Statistics from Copernicus Land Service that we have publicly stored in OpenStack Object storage (Swift).

In [ ]:
import s3fs
import xarray as xr

In [ ]:
fs = s3fs.S3FileSystem(anon=True,
      client_kwargs={
         'endpoint_url': 'https://object-store.cloud.muni.cz'
      })

:::{tip}
The parameter `anon` is for `anonymous` and is set to `True` because the data we have stored at `https://object-store.cloud.muni.cz` is public
:::

#### List files and folders in existing buckets

Instead of organizing files in various folders, object storage systems store files in a flat organization of containers (called "buckets"). 

In [ ]:
fs.ls('foss4g-data')

In [ ]:
fs.ls('foss4g-data/CGLS_LTS_1999_2019_Lombardia')

#### Access remote files from S3-compatible Object Storage

In [ ]:
s3path = 's3://foss4g-data/CGLS_LTS_1999_2019_Lombardia/AOI_c_gls_NDVI-LTS_1999-2019-0101_GLOBE_VGT-PROBAV_V3.0.1.nc'

In [ ]:
LTS = xr.open_dataset(fs.open(s3path))

In [ ]:
LTS

In [ ]:
LTS.sel(lat=45.88, lon=8.63, method='nearest')['min'].values

:::{warning}
The same dataset can be available from different locations e.g. [CGLS distributor VITO](https://vito.be/en), Zenodo, S3-compatible OpenStack Object storage (Swift), etc.
How do you know if it corresponds to the very same dataset? You cannot know except if the datasets have a persistent identifier such as a Digital Object Identifier.
It is therefore recommended *1)* to be extra careful about where you get your datasets, and *2)* to double check that the content is exactly what you expect (for instance, you can perform basic quality checks).                                                            
:::

#### Access multiple remote files 

In [ ]:
s3path = 's3://foss4g-data/CGLS_LTS_1999_2019_Lombardia/AOI_c_gls_*.nc'

In [ ]:
remote_files = fs.glob(s3path)
remote_files

We need to add a time dimension to concatenate data. For this, we define a function that will be called for each remote file (via the `preprocess` parameter of Xarray `open_mfdataset`.)

In [ ]:
from datetime import datetime

In [ ]:
def preprocess(ds):
    t = datetime.strptime(ds.attrs['identifier'].split(':')[-1].split('_')[1].replace('1999-', ''), "%Y-%m%d")
    return(ds.assign_coords(time=t).expand_dims('time'))   

Xarray `open_mfdataset` allows to open multiple files at the same time.

In [ ]:
# Iterate through remote_files to create a fileset
fileset = [fs.open(file) for file in remote_files]

When opening remote files, you can also select the variables you wish to analyze.

In [ ]:
LTS = xr.open_mfdataset(fileset, combine='nested', concat_dim=['time'],  preprocess=preprocess,
                        decode_coords="all")
LTS

:::{tip}
If you use one of xarray’s open methods such as xarray.open_dataset to load netCDF files with the default engine, it is recommended to use decode_coords=”all”. This will load the grid mapping variable into coordinates for compatibility with rioxarray. See [rioxarray documentation](https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html#xarray).
:::

## Preparing and discover online datasets

With the plethora of cloud storage, there are many available online datasets. To ease the preparation and discovery of such datasets, we describe emerging community-driven initiatives promoting standards suited to both geospatial and geoscience communities. Most of the material below is adapted from a previous Pangeo 101 training {cite:ps}`galaxy2022-pangeo`.

### Analysis Ready, cloud optimized data (ARCO)
When analyzing data at scale, the data format used is key. For years, the main data format was netCDF e.g. Network Common Data Form but with the use of cloud computing and interest in Open Science, different formats are often more suitable.

Formats for analyzing data from the cloud are refered to as "Analysis Ready, Cloud Optimized" data formats or in short ARCO. Find further info about ARCO datasets in {cite:ps}`Abernathey2022-arco`.

What is "Analysis Ready"?
* Think in terms of "Datasets" not "data files"
* No need for tedious homogenizing / cleaning setup guides
* Curated and cataloged

What is "Cloud Optimized"?
* Compatible with object storage e.g. access via HTTP
* Supports lazy access and intelligent subsetting
* Integrates with high-level analysis libraries and distributed frameworks

Instead of having a big dataset, ARCO datasets are chunked appropriately for analysis and have rich metadata (See Figure 1).

<img src="https://github.com/galaxyproject/training-material/blob/696dfecd4c88e59b487a7a3557cfedca6ec5754b/topics/climate/images/arco_data.png?raw=true" align="Left" /></a>

*Fig 1. Example of an ARCO dataset. Source: {cite:ps}`galaxy2022-pangeo`.*

### The Pangeo forge initiative

[Pangeo Forge](https://pangeo-forge.org/) is an open source platform for data **Extraction**, **Transformation**, and **Loading** (ETL). The goal of *Pangeo Forge* is to make it easy to extract data from traditional repositories and deposit this data in cloud object storage in an analysis-ready, cloud optimized (ARCO) format {cite:ps}`galaxy2022-pangeo`.

Pangeo Forge is inspired directly by Conda Forge, a community-led collection of recipes for building conda packages.

It is under active development and the Pangeo community hopes it will play a role in democratizing the publication of datasets in ARCO format.

#### How does Pangeo Forge work?

Pangeo Forge defines the concept of a recipe, which specifies the logic for transforming a specific data archive into an ARCO data store.
All contributions to Pangeo Forge must include an executable Python module, named recipe.py or similar, in which the data transformation logic is embedded (Figure 2).
The recipe contributor is expected to use one of a predefined set of template algorithms defined by Pangeo Forge.
Each of these templated algorithms is designed to transform data of a particular source type into a corresponding ARCO format, and requires only that the contributor populate the template with information unique to their specific data transformation, including the location of the source files and the way in which they should be aligned in the resulting ARCO data store {cite:ps}`Abernathey2022-arco`.

The diagram below looks complicated but like for conda forge most of the process is automated.

<img src="https://www.frontiersin.org/files/Articles/782909/fclim-03-782909-HTML/image_m/fclim-03-782909-g002.jpg" align="Left" /></a>

*Fig 2. A recipe in relation to Pangeo Forge architecture. Source: {cite:ps}`Abernathey2022-arco`.*

The next step after preparing the dataset is then to tell the community where and how to access to your transformed dataset.

This is done by creating a catalog.

### Spatio Temporal Asset Catalogs (STAC)

The [STAC](https://stacspec.org/en/) specification is a common language to describe geospatial information, so it can more easily be worked with, indexed, and discovered.

#### Why STAC?
* Each provider has its own catalog and interface (APIs).
* Every time you want to access a new catalog, you need to change your program.
* We have lots of data providers and each with a bespoke interface.
* It is becoming quickly difficult for programmers who need to design a new data connector each time.

#### Features
- STAC catalogs are extremely simple.
- They are composed of three layers:
    - **Catalogs**
        - **Collections**
            - **Items**
- STAC is very popular for Earth Observation satellite imagery.
- For instance it can be used to access Sentinel-2 in AWS (see Figure 3).

<img src="https://github.com/galaxyproject/training-material/blob/696dfecd4c88e59b487a7a3557cfedca6ec5754b/topics/climate/images/sentinel2_AWS.png?raw=true" align="Left" /></a>

*Fig 1. Example of STAC collection of Sentinel-2 images hosted in AWS. 
Source: {cite:ps}`galaxy2022-pangeo`.*


#### STAC and Pangeo Forge
- Pangeo-forge supports the creation of analysis-ready cloud optimized (ARCO) data in cloud object storage from "classical" data repositories'
- STAC is used to create catalogs and goes beyond the Pangeo ecosystem.
- Work is ongoing to figure out the best way to expose Pangeo-Forge-generated data assets via STAC catalogs.

<div class="alert alert-success">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Key Points</b>
    <br>
    <ul>
        <li>Access to remote dataset</li>
        <li>ARCO datasets</li>
        <li>Pangeo Forge</li>
        <li>STAC</li>
    </ul>
</div>

## References

```{bibliography}
:style: alpha
:filter: topic % "data-access"
```